# CrewAI Middle Level: Multi-Agent Collaboration

## Building Effective AI Teams

In this notebook, you'll learn how to create teams of agents that work together effectively.

### What You'll Learn:
1. Creating multiple specialized agents
2. Using built-in tools
3. Task dependencies and delegation
4. Process types: Sequential vs Hierarchical
5. Agent collaboration and communication

### Prerequisites:
- Completed Entry Level notebook
- Understanding of Agent, Task, and Crew concepts

---

## Setup

In [8]:
# Install packages (run once)
%pip install crewai crewai-tools python-dotenv --quiet

In [9]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# ============================================
# GLOBAL CONFIGURATION
# ============================================
MODEL = "gpt-5-mini"

# Import CrewAI components
from crewai import Agent, Task, Crew, Process, LLM

# Initialize LLM
llm = LLM(model=f"openai/{MODEL}", temperature=0.7)

print(f"Environment loaded. Using model: {MODEL}")

Environment loaded. Using model: gpt-4o-mini


## Part 1: Creating a Multi-Agent Team

Let's create a content creation team with three specialized agents:
- **Researcher**: Gathers information
- **Writer**: Creates content
- **Editor**: Refines and polishes

In [10]:
# Agent 1: The Researcher
researcher = Agent(
    role="Senior Research Analyst",
    goal="Uncover cutting-edge developments and key insights on given topics",
    backstory="""You are a seasoned research analyst with a keen eye for detail.
    You excel at finding relevant information and synthesizing complex data
    into actionable insights. Your research forms the foundation of quality content.""",
    llm=llm,
    verbose=True,
    allow_delegation=False  # This agent focuses only on research
)

# Agent 2: The Writer
writer = Agent(
    role="Content Strategist",
    goal="Craft compelling content based on research findings",
    backstory="""You are a creative content strategist who transforms research
    into engaging narratives. You understand audience needs and create content
    that educates and inspires. Your writing is clear, concise, and impactful.""",
    llm=llm,
    verbose=True,
    allow_delegation=False
)

# Agent 3: The Editor
editor = Agent(
    role="Senior Editor",
    goal="Ensure content is polished, accurate, and publication-ready",
    backstory="""You are a meticulous editor with years of experience in publishing.
    You have a sharp eye for grammar, style consistency, and factual accuracy.
    You ensure every piece meets the highest quality standards.""",
    llm=llm,
    verbose=True,
    allow_delegation=False
)

print("Team created successfully!")
print(f"Team members: {researcher.role}, {writer.role}, {editor.role}")

Team created successfully!
Team members: Senior Research Analyst, Content Strategist, Senior Editor


## Part 2: Task Dependencies

Tasks can be connected using the `context` parameter. This allows outputs from previous tasks to flow into subsequent tasks.

In [11]:
# Define the topic for our content pipeline
TOPIC = "The future of AI in healthcare"

# Task 1: Research (no dependencies)
research_task = Task(
    description=f"""Conduct comprehensive research on: {TOPIC}
    
    Focus on:
    1. Current applications and use cases
    2. Recent breakthroughs and innovations
    3. Challenges and limitations
    4. Future predictions and trends
    
    Provide detailed findings with specific examples.""",
    expected_output="""A comprehensive research report with:
    - Executive summary
    - Key findings organized by category
    - Specific examples and case studies
    - Sources and references""",
    agent=researcher
)

# Task 2: Writing (depends on research)
writing_task = Task(
    description=f"""Using the research provided, write an engaging article on: {TOPIC}
    
    Requirements:
    - Write for a professional but accessible audience
    - Include an attention-grabbing introduction
    - Organize content with clear sections
    - Include practical implications""",
    expected_output="""A well-structured article (500-700 words) with:
    - Compelling headline
    - Engaging introduction
    - 3-4 main sections with insights
    - Strong conclusion with takeaways""",
    agent=writer,
    context=[research_task]  # This task uses output from research_task
)

# Task 3: Editing (depends on writing)
editing_task = Task(
    description="""Review and edit the article to ensure:
    
    1. Grammar and spelling are perfect
    2. Flow and readability are excellent
    3. Key messages are clear
    4. Tone is consistent throughout
    5. Facts are accurately represented""",
    expected_output="""A polished, publication-ready article with:
    - All errors corrected
    - Improved clarity and flow
    - Consistent tone and style
    - Editorial notes on changes made""",
    agent=editor,
    context=[writing_task]  # This task uses output from writing_task
)

print("Task pipeline created!")
print("Flow: Research → Writing → Editing")

Task pipeline created!
Flow: Research → Writing → Editing


In [12]:
# Create a crew with SEQUENTIAL process
sequential_crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research_task, writing_task, editing_task],
    process=Process.sequential,  # Tasks run in order
    verbose=True
)

print("Sequential Crew created!")
print("Tasks will execute: research_task → writing_task → editing_task")

Sequential Crew created!
Tasks will execute: research_task → writing_task → editing_task


In [13]:
# Execute the sequential crew
result = sequential_crew.kickoff()

print("\n" + "="*60)
print("CONTENT PIPELINE COMPLETE!")
print("="*60)
print("\nFinal Article:")
print(result)

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  776a976a-5908-4a37-b4ed-e4ce991e23da                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Conduct comprehensive research on: The future of AI in healthcare                                        │
│                                                                                                                 │
│      Focus on:                                                                                                  │
│      1. Current applications and use cases                                                                      │
│      2. Recent breakthroughs and innovations                                                                    │
│      3. Challenges and limitations                                                                              │
│      4. Future predictions and trends                                                                           │
│                                                                                                                 │
│      Provide detailed findings with specific examples.                                                          │
│  ID: 1fb69174-2cf1-491d-99c6-65385ff268bc                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Conduct comprehensive research on: The future of AI in healthcare                                        │
│                                                                                                                 │
│      Focus on:                                                                                                  │
│      1. Current applications and use cases                                                                      │
│      2. Recent breakthroughs and innovations                                                                    │
│      3. Challenges and limitations                                                                              │
│      4. Future predictions and trends                                                                           │
│                                                                                                                 │
│      Provide detailed findings with specific examples.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Research Report: The Future of AI in Healthcare                                                              │
│                                                                                                                 │
│  ## Executive Summary                                                                                           │
│  The integration of Artificial Intelligence (AI) in healthcare is transforming patient care, diagnostics, and   │
│  operational efficiencies. Current applications range from predictive analytics and personalized medicine to    │
│  automated administrative tasks. Recent breakthroughs, including advanced machine learning algorithms and       │
│  natural language processing, are enhancing decision-making processes. However, challenges such as data         │
│  privacy, ethical considerations, and the need for regulatory frameworks remain significant. Future trends      │
│  suggest a shift towards more collaborative AI-human interactions, increased adoption of AI in telemedicine,    │
│  and innovations in drug discovery and genomics. This report provides a comprehensive overview of the current   │
│  landscape, innovations, challenges, and future predictions for AI in healthcare.                               │
│                                                                                                                 │
│  ## Key Findings                                                                                                │
│                                                                                                                 │
│  ### 1. Current Applications and Use Cases                                                                      │
│  AI technologies are already being utilized in various aspects of healthcare:                                   │
│                                                                                                                 │
│  - **Diagnostics**: AI algorithms are being deployed in radiology to analyze medical images. For instance,      │
│  Google's DeepMind developed an AI system that can detect over 50 eye diseases with accuracy comparable to      │
│  human specialists.                                                                                             │
│                                                                                                                 │
│  - **Predictive Analytics**: Tools like IBM Watson Health leverage AI to analyze patient data and predict       │
│  health outcomes, allowing for early interventions. For example, the system can assess the likelihood of        │
│  hospital readmission for patients with chronic conditions.                                                     │
│                                                                                                                 │
│  - **Personalized Medicine**: AI is being used to tailor treatment plans based on individual genetic profiles.  │
│  Tempus, a technology company, utilizes AI to analyze clinical and molecular data to support oncologists in     │
│  developing personalized cancer therapies.                                                                      │
│                                                                                                                 │
│  - **Virtual Health Assistants**: AI-powered chatbots a

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Conduct comprehensive research on: The future of AI in healthcare                                              │
│                                                                                                                 │
│      Focus on:                                                                                                  │
│      1. Current applications and use cases                                                                      │
│      2. Recent breakthroughs and innovations                                                                    │
│      3. Challenges and limitations                                                                              │
│      4. Future predictions and trends                                                                           │
│                                                                                                                 │
│      Provide detailed findings with specific examples.                                                          │
│  Agent:                                                                                                         │
│  Senior Research Analyst                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Editor                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # The Future of AI in Healthcare: Transforming Patient Care and Beyond                                         │
│                                                                                                                 │
│  The healthcare landscape is on the brink of a revolution, one defined by the integration of Artificial         │
│  Intelligence (AI) into its core processes. As we stand at the intersection of technology and medicine, the     │
│  potential for AI to enhance patient care, streamline operations, and facilitate groundbreaking innovations is  │
│  more promising than ever. This article delves into the current applications of AI in healthcare, recent        │
│  breakthroughs, existing challenges, and predictions for the future, providing a comprehensive view of how AI   │
│  is set to transform the industry.                                                                              │
│                                                                                                                 │
│  ## Current Applications of AI in Healthcare                                                                    │
│                                                                                                                 │
│  AI technologies are already making significant inroads into various facets of healthcare.                      │
│                                                                                                                 │
│  ### Diagnostics                                                                                                │
│                                                                                                                 │
│  One of the most impactful applications of AI is in diagnostics. Advanced AI algorithms, such as those          │
│  developed by Google’s DeepMind, can analyze medical images with remarkable accuracy—detecting over 50 eye      │
│  diseases at a level comparable to that of human specialists. This capability not only enhances diagnostic      │
│  precision but also expedites the identification of conditions that require immediate attention.                │
│                                                                                                                 │
│  ### Predictive Analytics                                                                                       │
│                                                                                                                 │
│  Predictive analytics is another critical area where AI shines. Tools like IBM Watson Health leverage vast      │
│  datasets to analyze patient histories and predict health outcomes, enabling healthcare providers to intervene  │
│  early in chronic conditions. For instance, by assessing the likelihood of hospital readmission, healthcare     │
│  teams can tailor their follow-up care strategies, ultimately improving patient outcomes and reducing costs.    │
│                                                                                                                 │
│  ### Personalized Medicine                                                                                      │
│                                                                                                                 │
│  In the realm of personalized medicine, AI is revolutio

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Review and edit the article to ensure:                                                                         │
│                                                                                                                 │
│      1. Grammar and spelling are perfect                                                                        │
│      2. Flow and readability are excellent                                                                      │
│      3. Key messages are clear                                                                                  │
│      4. Tone is consistent throughout                                                                           │
│      5. Facts are accurately represented                                                                        │
│  Agent:                                                                                                         │
│  Senior Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  776a976a-5908-4a37-b4ed-e4ce991e23da                                                                           │
│  Final Output: # The Future of AI in Healthcare: Transforming Patient Care and Beyond                           │
│                                                                                                                 │
│  The healthcare landscape is on the brink of a revolution, one defined by the integration of Artificial         │
│  Intelligence (AI) into its core processes. As we stand at the intersection of technology and medicine, the     │
│  potential for AI to enhance patient care, streamline operations, and facilitate groundbreaking innovations is  │
│  more promising than ever. This article delves into the current applications of AI in healthcare, recent        │
│  breakthroughs, existing challenges, and predictions for the future, providing a comprehensive view of how AI   │
│  is set to transform the industry.                                                                              │
│                                                                                                                 │
│  ## Current Applications of AI in Healthcare                                                                    │
│                                                                                                                 │
│  AI technologies are already making significant inroads into various facets of healthcare.                      │
│                                                                                                                 │
│  ### Diagnostics                                                                                                │
│                                                                                                                 │
│  One of the most impactful applications of AI is in diagnostics. Advanced AI algorithms, such as those          │
│  developed by Google’s DeepMind, can analyze medical images with remarkable accuracy—detecting over 50 eye      │
│  diseases at a level comparable to that of human specialists. This capability not only enhances diagnostic      │
│  precision but also expedites the identification of conditions that require immediate attention.                │
│                                                                                                                 │
│  ### Predictive Analytics                                                                                       │
│                                                                                                                 │
│  Predictive analytics is another critical area where AI shines. Tools like IBM Watson Health leverage vast      │
│  datasets to analyze patient histories and predict health outcomes, enabling healthcare providers to intervene  │
│  early in chronic conditions. For instance, by assessing the likelihood of hospital readmission, healthcare     │
│  teams can tailor their follow-up care strategies, ultimately improving patient outcomes and reducing costs.    │
│                                                                                                                 │
│  ### Personalized Medicine                            


CONTENT PIPELINE COMPLETE!

Final Article:
# The Future of AI in Healthcare: Transforming Patient Care and Beyond

The healthcare landscape is on the brink of a revolution, one defined by the integration of Artificial Intelligence (AI) into its core processes. As we stand at the intersection of technology and medicine, the potential for AI to enhance patient care, streamline operations, and facilitate groundbreaking innovations is more promising than ever. This article delves into the current applications of AI in healthcare, recent breakthroughs, existing challenges, and predictions for the future, providing a comprehensive view of how AI is set to transform the industry.

## Current Applications of AI in Healthcare

AI technologies are already making significant inroads into various facets of healthcare. 

### Diagnostics

One of the most impactful applications of AI is in diagnostics. Advanced AI algorithms, such as those developed by Google’s DeepMind, can analyze medical images w

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Part 4: Using Built-in Tools

CrewAI provides various tools that extend agent capabilities. Let's explore some common ones.

In [14]:
from crewai_tools import (
    SerperDevTool,      # Web search
    FileReadTool,       # Read files
    DirectoryReadTool,  # Read directories
)

# Note: Some tools require API keys
# SerperDevTool requires SERPER_API_KEY in .env

# Example: Creating a file reading tool
file_reader = FileReadTool()

# Example: Creating a directory reading tool
directory_reader = DirectoryReadTool()

print("Tools loaded successfully!")
print("Available tools: FileReadTool, DirectoryReadTool")
print("\nNote: SerperDevTool requires SERPER_API_KEY for web search")

Tools loaded successfully!
Available tools: FileReadTool, DirectoryReadTool

Note: SerperDevTool requires SERPER_API_KEY for web search


In [15]:
# Create an agent with tools
research_agent_with_tools = Agent(
    role="Technical Researcher",
    goal="Research and analyze technical documents and files",
    backstory="""You are a technical researcher skilled at analyzing 
    documents and extracting key information. You use available tools
    to gather and process information efficiently.""",
    llm=llm,
    tools=[file_reader, directory_reader],  # Assign tools to agent
    verbose=True
)

print(f"Agent created with {len(research_agent_with_tools.tools)} tools")

Agent created with 2 tools


## Part 5: Agent Delegation

When `allow_delegation=True`, agents can delegate tasks to other agents in the crew.

In [16]:
# Project Manager with delegation ability
project_manager = Agent(
    role="Project Manager",
    goal="Coordinate team efforts and ensure project success",
    backstory="""You are an experienced project manager who excels at
    coordinating teams and delegating tasks effectively. You understand
    each team member's strengths and assign work accordingly.""",
    llm=llm,
    allow_delegation=True,  # Can delegate to other agents
    verbose=True
)

# Specialist who does focused work
specialist = Agent(
    role="Technical Specialist",
    goal="Provide deep technical expertise on assigned topics",
    backstory="""You are a technical specialist with deep expertise
    in software development and AI. You focus on delivering high-quality
    technical work when called upon.""",
    llm=llm,
    allow_delegation=False,  # Focuses on own work
    verbose=True
)

print("Delegation example created!")
print(f"Project Manager can delegate: {project_manager.allow_delegation}")
print(f"Specialist can delegate: {specialist.allow_delegation}")

Delegation example created!
Project Manager can delegate: True
Specialist can delegate: False


## Practice Exercise: Build Your Own Multi-Agent Team

Create a customer support team with at least 3 agents!

In [17]:
# EXERCISE: Create a customer support team
# Suggestions: Intake Specialist, Technical Support, Customer Success Manager

# Agent 1: Intake Specialist
intake_agent = Agent(
    role="YOUR_ROLE",
    goal="YOUR_GOAL",
    backstory="YOUR_BACKSTORY",
    llm=llm,
    verbose=True
)

# Agent 2: Technical Support
# tech_support = Agent(...)

# Agent 3: Customer Success
# success_manager = Agent(...)

# Create tasks with dependencies
# intake_task = Task(...)
# support_task = Task(..., context=[intake_task])
# followup_task = Task(..., context=[support_task])

# Create and run the crew
# support_crew = Crew(
#     agents=[...],
#     tasks=[...],
#     process=Process.sequential,
#     verbose=True
# )
# result = support_crew.kickoff()
# print(result)

## Summary

In this notebook, you learned:
- How to create multi-agent teams with specialized roles
- Using task dependencies with the context parameter
- Process types: Sequential for linear workflows
- Adding tools to extend agent capabilities
- Delegation between agents

Next Steps: Move to the Advanced Level notebook to learn about Flows